In [1]:
import pandas as pd
import numpy as np

In [5]:
df = pd.read_csv('Ano-2019.csv', sep=';')
df2 = pd.read_csv('Ano-2020.csv', sep=';')
df3 = pd.read_csv('Ano-2021.csv', sep=';')

In [8]:
df2['cpf']

0                  NaN
1                  NaN
2                  NaN
3                  NaN
4                  NaN
              ...     
160898    4.392800e+10
160899    4.392800e+10
160900    4.392800e+10
160901    4.392800e+10
160902    4.392800e+10
Name: cpf, Length: 160903, dtype: float64